# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float
    
    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__


def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [2]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
import math

@dataclass
class Dual:
    value: float
    d: float
    
    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    __rmul__ = __mul__
    __radd__ = __add__

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)
    
    def __pos__(self) -> "Dual":
        return Dual(self.value, self.d)
    
    def __abs__(self) -> "Dual":
        return Dual(abs(self.value), self.value / abs(self.value) * self.d)
    
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)
    
    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return self.__sub__(other, self)
            case Number():
                return Dual(float(other) - self.value, -self.d)
    
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / o_value ** 2)
            case Number():
                return Dual(self.value / float(other), self.d / float(other))
    
    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return self.__truediv__(other, self)
            case Number():
                return Dual(float(other) / self.value, -float(other) * self.value ** -2 * self.d)

    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value ** o_value, o_value * self.value ** (o_value - 1) * self.d + self.value ** o_value * o_d * math.log(self.value))
            case Number():
                return Dual(self.value ** float(other), float(other) * self.value ** (float(other) - 1) * self.d)
    
    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return self.__pow__(other, self)
            case Number():
                return Dual(float(other) ** self.value, float(other) ** self.value * math.log(float(other)) * self.d)

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

In [2]:
tests_1 = [lambda x: -x * 5 + 3 * +x,
           lambda x: abs(x * 5),
           lambda x: (3 / (x * x)) + (x / x) - (x / 2),
           lambda x: x ** 3 - 2 ** x + x ** x,
           lambda x: abs(-x / 2) / (((2 ** x) ** x) ** (2 / +x))]

for f in tests_1:
    f_diff = diff(f)
    print(f_diff(2))

-2.0
5.0
-1.25
16.0
-0.05539339756999316


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [3]:
def exp(dual: "Dual") -> "Dual":
    return Dual(math.exp(dual.value), math.exp(dual.value) * dual.d)

def cos(dual: "Dual") -> "Dual":
    return Dual(math.cos(dual.value), -math.sin(dual.value) * dual.d)

def sin(dual: "Dual") -> "Dual":
    return Dual(math.sin(dual.value), math.cos(dual.value) * dual.d)

def log(dual: "Dual") -> "Dual":
    return Dual(math.log(dual.value), 1 / dual.value * dual.d)

In [4]:
tests_2 = [lambda x: exp(x ** 2),
           lambda x: cos(x / 3.4),
           lambda x: sin(x) ** 2,
           lambda x: -log(x / 10),
           lambda x: exp(sin(cos(x)) / log(x))]

for f in tests_2:
    f_diff = diff(f)
    print(f_diff(2))

218.39260013257694
-0.16320405631285667
-0.7568024953079283
-0.5
-0.4348774016278626


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [ ]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [5]:
from scipy.misc import derivative
import numpy as np

tests = [lambda x: -x * 5 + 3 * +x,
         lambda x: abs(x * 5),
         lambda x: (3 / (x * x)) + (x / x) - (x / 2),
         lambda x: x ** 3 - 2 ** x + x ** x,
         lambda x: abs(-x / 2) / (((2 ** x) ** x) ** (2 / +x)),
         lambda x: np.exp(x ** 2),
         lambda x: np.cos(x / 3.4),
         lambda x: np.sin(x) ** 2,
         lambda x: -np.log(x / 10),
         lambda x: np.exp(np.sin(np.cos(x)) / np.log(x))]

for f in tests:
    print(derivative(f, 2., dx=1e-6))

-2.000000000723645
5.00000000069889
-1.2500000001192113
16.00000000134827
-0.05539339757373862
218.3926001393388
-0.16320405632486157
-0.7568024952830932
-0.500000000069889
-0.4348774016449397


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [64]:
from random import randint, choice

def gen_func(var_number: int = 6) -> Callable[[float], float]:
    func = "lambda x: "

    un_op = ['+', '-']
    bi_op = [" + ", " - ", " * ", " / ", " ** "]
    math_functions = ["abs(", "exp(", "cos(", "sin(", "log("]

    scripts = ["var", "number", "un_op", "func", "open_br"]
    close_br_count = 0
    var_count = 0

    while(var_count < var_number):
        match choice(scripts):
            case "var":
                func += 'x'
                scripts = ["bi_op"] + ["close_br"] * close_br_count
                var_count += 1
            case "number":
                func += str(randint(0, 10))
                scripts = ["bi_op"] + ["close_br"] * close_br_count
            case "un_op":
                func += choice(un_op)
                scripts = ["var"] * 3 + ["number", "func", "open_br"]
            case "bi_op":
                func += choice(bi_op)
                scripts = ["var"] * 3 + ["number", "func", "open_br"]
            case "func":
                func += choice(math_functions)
                close_br_count += 1
                scripts = ["var"] * 3 + ["number", "un_op", "func", "open_br"]
            case "open_br":
                func += '('
                close_br_count += 1
                scripts = ["var"] * 3 + ["number", "un_op", "func"]
            case "close_br":
                func += ')'
                close_br_count -= 1
                scripts = ["bi_op"]

    if close_br_count > 0:
        func += ')' * close_br_count

    return eval(func)

f = gen_func()
f(5)

lambda x: 6 ** x * 6 + (x - x * x) / x - (x)


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [ ]:
# ваш код